## Snowflake Query

Note that there are some limitations in using the Snowflake connector from inside a container:
1. The first connection attempt after opening the container will occasionally hang at the browser redirection page. If this happens 'Interrupt' the execution and try again.
2. Sometimes during authentication, the browser will redirect to an error at a localhost page. This is caused by a race condition between the authentication and the container port forwarding. You can do as above and retry until it works. Alternatively you can open the `Terminal` panel, look at the `PORTS` tab and wait until the port is autoforwarded then retry the address in the browser.

Here we do a simple query to fetch data from Snowflake into a pandas dataframe using a helper function `query_to_pandas` for convinience.


In [ ]:
from np_exploration_python.snowflake import query_to_pandas

df = query_to_pandas("select * from marketing.np_gth_features_leg limit 1000")

df

## Lower-level Snowflake Access

We can also work with the [Snowflake Python Connector API](https://docs.snowflake.com/en/user-guide/python-connector-api.html#) directly when low-level control is needed. For example, in order to:
- Change the connection properties.
- Use cursors to execute SQL statements other than queries.
- Reuse the connection (`ctx` in this example) to execute multiple sql statements.
- Query larger sets of data in batches by using [fetch_pandas_batches()](https://docs.snowflake.com/en/user-guide/python-connector-api.html#fetch_pandas_batches) instead of fetch_pandas_all().

Here we use the API to execute the same query as in the cell above.

In [ ]:
import snowflake.connector
from np_exploration_python.vscode_settings import VSCodeSettings


config_dict = VSCodeSettings().get_snowflake_connection_config()
ctx = snowflake.connector.connect(**config_dict)

cs = ctx.cursor()
try:
    # Execute a statement that will generate a result set.
    sql = "select * from marketing.np_gth_features_leg limit 1000"
    cs.execute(sql)

    # Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
    df = cs.fetch_pandas_all()
finally:
    cs.close()
ctx.close()

df.head()

Pulling down a large dataset can be time consuming so you may want to cache the data locally for later use.

Here we dump the features to a compressed file.

In [ ]:
import pyarrow.feather as feather

feather.write_feather(df, 'feature.data')
del df

Simple data access using cached data when available.

In [ ]:
from pathlib import Path
from np_exploration_python.snowflake import query_to_pandas

if Path('feature.data').is_file():
    print("Reading cached data")
    df = feather.read_feather('feature.data')
else:
    print("Connecting to data warehouse")
    df = query_to_pandas("select * from marketing.np_gth_features_leg limit 1000")

df.head()